In [ ]:
# Initial TB classifier with 3 classes

import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
classes = ["normal", "other_tb", "ptb"]

In [ ]:
import tensorflow as tf

In [ ]:
args = {
    "labels": "inferred",
    "label_mode": "categorical",
    "batch_size": 32,
    "image_size": (256, 256),
    "seed": 1,
    "validation_split": .2,
    "class_names": classes
}

In [ ]:
train = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/classifier_input",
    subset="training",
    **args
)

test = tf.keras.utils.image_dataset_from_directory(
  "/content/drive/MyDrive/classifier_input",
  subset="validation",
    **args
)

In [ ]:
first = train.take(1)
first

In [ ]:
images, labels = list(first)[0]
first_image = images[0]

In [ ]:
first_image[:3,:3,0]

In [ ]:
from PIL import Image

Image.fromarray(images[0].numpy().astype("uint8"))

In [ ]:
train = train.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test = test.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
# Making a simple sequential model
model = Sequential([
  tf.keras.layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(256,256,3)),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(len(classes))
])

In [ ]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train,
    validation_data=test,
    epochs=10,
    verbose=1
)

In [ ]:
model.summary()

In [ ]:
import pandas as pd

history_df = pd.DataFrame.from_dict(history.history)
history_df[["accuracy", "val_accuracy"]].plot()


In [ ]:
def train_model(network, epochs=15):
    model = Sequential(network)

    model.compile(optimizer='sgd',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    history = model.fit(
      train,
      validation_data=test,
      epochs=epochs
    )
    history_df = pd.DataFrame.from_dict(history.history)
    return history_df, model

In [ ]:
network = [
  tf.keras.layers.Rescaling(1./255),
  layers.Conv2D(16, 4, padding='same', activation='relu', input_shape=(256,256,3)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(len(classes))
]

history_df, model = train_model(network)

In [ ]:
history_df[["accuracy", "val_accuracy"]].plot()

In [ ]:
model.summary()

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    layers.RandomFlip("vertical", seed=1),
      layers.RandomRotation(0.2, seed=1),
    layers.RandomZoom(0.2, seed=1),
  ]
)


full_network = [
    data_augmentation
] + network

history_df, model = train_model(full_network, epochs=40)

In [ ]:
history_df[["accuracy", "val_accuracy"]].plot()

In [ ]:
model.summary()

In [ ]:
preds = model.predict(test)

In [ ]:
import numpy as np
predicted_class = np.argmax(preds, axis=1)

In [ ]:
actual_labels = np.concatenate([y for x, y in test], axis=0)

In [ ]:
import itertools

actual_image = [x.numpy().astype("uint8") for x, y in test]
actual_image = list(itertools.chain.from_iterable(actual_image))
actual_image = [Image.fromarray(a) for a in actual_image]

In [ ]:
actual_class = np.argmax(actual_labels, axis=1)

In [ ]:
pred_df = pd.DataFrame(zip(predicted_class, actual_class, actual_image), columns=["prediction", "actual", "image"])

In [ ]:
pred_df["prediction"] = pred_df["prediction"].apply(lambda x: classes[x])
pred_df["actual"] = pred_df["actual"].apply(lambda x: classes[x])

In [ ]:
import base64
import io

def image_formatter(img):
    with io.BytesIO() as buffer:
        img.save(buffer, 'png')
        img_str = base64.b64encode(buffer.getvalue()).decode()
        return f'{img_str}">'

pred_df.head(10).style.format({'image': image_formatter})

In [ ]:
from sklearn import metrics
# Predicted values
y_pred = pred_df["prediction"]
# Actual values
y_act = pred_df["actual"]
# Printing the confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
print(metrics.confusion_matrix(y_act, y_pred, labels=["normal", "other_tb", "ptb"]))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=["normal", "other_tb", "ptb"]))